In [103]:
import pandas as pd

Cargar el fichero y ver que contiene

In [104]:
datos = pd.read_csv('../dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')
datos.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [105]:
datos.Churn = (datos.Churn == 'Yes').astype(int)
datos.Churn.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

Comienza el procesado de la información. Para ello definimos objetivos y conjuntos de datos de entrenamiento y de validación.

In [106]:
from sklearn.model_selection import train_test_split
datos_train_full, datos_test = train_test_split(datos, test_size=0.2, random_state=1)

datos_train, datos_val = train_test_split(datos_train_full, test_size=0.33, random_state=1)
y_train = datos_train.Churn.values
y_val = datos_val.Churn.values
y_test= datos_test.Churn.values
del datos_train['Churn']
del datos_val['Churn']
del datos_test['Churn']

datos_train.head().T

,4204,7034,5146,5184,1310
customerID,4395-PZMSN,0639-TSIQW,3797-FKOGQ,7570-WELNY,6393-WRYZE
gender,Male,Female,Male,Female,Female
SeniorCitizen,1,0,0,0,0
Partner,No,No,No,Yes,Yes
Dependents,No,No,Yes,No,No
tenure,5,67,11,68,34
PhoneService,Yes,Yes,Yes,Yes,Yes
MultipleLines,No,Yes,Yes,Yes,Yes
InternetService,Fiber optic,Fiber optic,Fiber optic,Fiber optic,Fiber optic
OnlineSecurity,No,Yes,No,Yes,No


In [107]:
train_dict = datos_train.to_dict(orient='records')
dict(sorted(train_dict[0].items()))

{'Contract': 'Month-to-month',
 'Dependents': 'No',
 'DeviceProtection': 'No',
 'InternetService': 'Fiber optic',
 'MonthlyCharges': 85.55,
 'MultipleLines': 'No',
 'OnlineBackup': 'Yes',
 'OnlineSecurity': 'No',
 'PaperlessBilling': 'Yes',
 'Partner': 'No',
 'PaymentMethod': 'Electronic check',
 'PhoneService': 'Yes',
 'SeniorCitizen': 1,
 'StreamingMovies': 'Yes',
 'StreamingTV': 'No',
 'TechSupport': 'No',
 'TotalCharges': '408.5',
 'customerID': '4395-PZMSN',
 'gender': 'Male',
 'tenure': 5}

In [108]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [109]:
X_train = dv.transform(train_dict)
X_train[0]

array([1., 0., 0., ..., 0., 1., 5.])

In [110]:
dv.get_feature_names_out()

array(['Contract=Month-to-month', 'Contract=One year',
       'Contract=Two year', ..., 'gender=Female', 'gender=Male', 'tenure'],
      dtype=object)

Entrenamiento del modelo

In [111]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [112]:
val_dict = datos_val.to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)
y_pred

array([[0.99162523, 0.00837477],
       [0.86457072, 0.13542928],
       [0.87386799, 0.12613201],
       ...,
       [0.37588096, 0.62411904],
       [0.83140748, 0.16859252],
       [0.8867945 , 0.1132055 ]])

In [113]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.00837477, 0.13542928, 0.12613201, ..., 0.62411904, 0.16859252,
       0.1132055 ])

In [114]:
churn = y_pred >= 0.5
churn

array([False, False, False, ...,  True, False, False])

In [115]:
round((y_val == churn).mean(), 3) 

0.808

In [116]:
test_dict = datos_test.to_dict(orient='records')
X_test = dv.transform(test_dict)
y_pred_test = model.predict_proba(X_test)
y_pred_test

array([[0.9385929 , 0.0614071 ],
       [0.93076102, 0.06923898],
       [0.69377118, 0.30622882],
       ...,
       [0.99243691, 0.00756309],
       [0.82294351, 0.17705649],
       [0.32568249, 0.67431751]])

In [117]:
y_pred_test = model.predict_proba(X_test)[:, 1]
y_pred_test

array([0.0614071 , 0.06923898, 0.30622882, ..., 0.00756309, 0.17705649,
       0.67431751])

In [118]:
churn = y_pred_test >= 0.67
churn

array([False, False, False, ..., False, False,  True])

In [119]:
round((y_test == churn).mean(), 3) 

0.801